# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.409039  
 0.640635  
 0.93061   
 0.958941  
 0.437158  
 0.420882  
 0.141985  
 0.662537  
 0.848262  
 0.663513  
 0.530988  
 0.271098  
 0.172754  
 ⋮         
 0.194441  
 0.170904  
 0.00288544
 0.347087  
 0.627563  
 0.278758  
 0.15979   
 0.793045  
 0.902089  
 0.945089  
 0.44942   
 0.837613  

In [2]:
sum(a)

5.000980033933877e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [3]:
#Pkg.add("BenchmarkTools")

In [4]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [5]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [6]:
c_sum(a)

5.000980033933847e6

In [7]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [8]:
c_sum(a) - sum(a)  

-2.9802322387695312e-8

In [9]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [10]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0, nans::Bool=false, norm::Function)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true
```


We can now benchmark the C code directly from Julia:

In [11]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     17.961 ms (0.00% GC)
  median time:      18.197 ms (0.00% GC)
  mean time:        18.192 ms (0.00% GC)
  maximum time:     18.502 ms (0.00% GC)
  --------------
  samples:          275
  evals/sample:     1

In [12]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

C: Fastest time was 17.961266 msec


In [13]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "C" => 17.9613

In [14]:
using Plots
gr()

LoadError: [91mArgumentError: Module Plots not found in current path.
Run `Pkg.add("Plots")` to install the Plots package.[39m

In [15]:
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

LoadError: [91mUndefVarError: histogram not defined[39m

# 2. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [16]:
#Pkg.add("PyCall")

In [17]:
using PyCall

In [18]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [19]:
pysum(a)

5.000980033933847e6

In [20]:
pysum(a) ≈ sum(a)

true

In [21]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     113.489 ms (0.00% GC)
  median time:      113.577 ms (0.00% GC)
  mean time:        115.027 ms (0.00% GC)
  maximum time:     149.832 ms (0.00% GC)
  --------------
  samples:          44
  evals/sample:     1

In [22]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "C"               => 17.9613
  "Python built-in" => 113.489

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [23]:
using Conda 
#Conda.add("numpy")

In [24]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

LoadError: [91mPyError (ccall(@pysym(:PyImport_ImportModule), PyPtr, (Cstring,), name)

The Python package numpy could not be found by pyimport. Usually this means
that you did not install numpy in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

python

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the numpy module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg

In [25]:
numpy_sum(apy_list) # python thing

LoadError: [91mUndefVarError: numpy_sum not defined[39m

In [26]:
numpy_sum(apy_list) ≈ sum(a)

LoadError: [91mUndefVarError: numpy_sum not defined[39m

In [27]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

LoadError: [91mUndefVarError: py_numpy_bench not defined[39m

# 4. Python, hand-written 

In [28]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7f10c8a58b18>

In [29]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     560.858 ms (0.00% GC)
  median time:      561.166 ms (0.00% GC)
  mean time:        563.097 ms (0.00% GC)
  maximum time:     576.253 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [30]:
@which sum([1.5])

sum(a) in Base at reduce.jl:359

In [31]:
ccall(:sleep, Void, (Cint,), 3)

In [32]:
Cint, Csize_t

(Int32, UInt64)

In [33]:
sum_py(apy_list)

5.000980033933847e6

In [34]:
sum_py(apy_list) ≈ sum(a)

true

In [35]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "C"                   => 17.9613
  "Python hand-written" => 560.858
  "Python built-in"     => 113.489

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [36]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [37]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.085 ms (0.00% GC)
  median time:      7.120 ms (0.00% GC)
  mean time:        7.122 ms (0.00% GC)
  maximum time:     7.396 ms (0.00% GC)
  --------------
  samples:          701
  evals/sample:     1

In [38]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                   => 17.9613
  "Python hand-written" => 560.858
  "Python built-in"     => 113.489
  "Julia built-in"      => 7.08527

# 6. Julia (hand-written) 

In [39]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A   
        s += a   
    end
    s
end

mysum (generic function with 1 method)

In [40]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     18.203 ms (0.00% GC)
  median time:      18.823 ms (0.00% GC)
  mean time:        18.792 ms (0.00% GC)
  maximum time:     19.351 ms (0.00% GC)
  --------------
  samples:          266
  evals/sample:     1

In [41]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                   => 17.9613
  "Julia hand-written"  => 18.2033
  "Python hand-written" => 560.858
  "Python built-in"     => 113.489
  "Julia built-in"      => 7.08527

# Summary

In [42]:
for (key, value) in sort(collect(d))
    println(rpad(key, 20, "."), lpad(round(value, 1), 8, "."))
end

C.......................18.0
Julia built-in...........7.1
Julia hand-written......18.2
Python built-in........113.5
Python hand-written....560.9


In [43]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............7.09
C........................17.96
Julia hand-written........18.2
Python built-in.........113.49
Python hand-written.....560.86
